In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FNS_Dataset_2023/training/traindata2.csv')

In [ ]:
df.head()

,Unnamed: 0,report,summary_2,summary_3,summary_4,summary_1,summary_5,summary_6,summary_7,best_summary_index,best_summary_score
0,0,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,NaN,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,NaN,NaN,2,0.935185
1,1,Synergy Health plc \nAnnual Report and Accou...,2010 2011 2012 2013 2014\n286.4 287.3\n312.0\...,Synergy remains well placed \nto exploit its...,1\n2\n3\n4\n1. UK & Ireland £164.7m\n2. Europ...,Strategic report \nChief Executive's strategi...,NaN,NaN,NaN,2,1.265226
2,2,Shanks Group plc Annual Report and Accounts 2...,2 shanks. annual report 2008\nfinancial highl...,shanks. annual report 2008 3\nchairman’s stat...,NaN,4 shanks. annual report 2008\nI joined as Gro...,NaN,NaN,NaN,3,0.707809
3,3,Annual Report & Accounts\n2012 Contents\nWynn...,1 Wynnstay Group Plc Annual Report & Accounts...,6 www.wynnstay.co.uk\nOVERVIEW\nIn my first s...,4 www.wynnstay.co.uk\nOur business at a glanc...,10 www.wynnstay.co.uk\nINTRODUCTION \nThe Gro...,NaN,NaN,NaN,2,1.131579
4,4,HSBC Holdings plc \nAnnual Report and Account...,HSBC HOLDINGS PLC \nReport of the Directors: ...,HSBC HOLDINGS PLC \nReport of the Directors: ...,NaN,7\nOverview Operating & Financial Review Gove...,NaN,NaN,NaN,2,0.721190


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3047 non-null   int64  
 1   report              3047 non-null   object 
 2   summary_2           3043 non-null   object 
 3   summary_3           3042 non-null   object 
 4   summary_4           735 non-null    object 
 5   summary_1           3040 non-null   object 
 6   summary_5           127 non-null    object 
 7   summary_6           8 non-null      object 
 8   summary_7           1 non-null      object 
 9   best_summary_index  3047 non-null   int64  
 10  best_summary_score  3047 non-null   float64
dtypes: float64(1), int64(2), object(8)
memory usage: 262.0+ KB


In [ ]:
df.drop(['Unnamed: 0','summary_4','summary_5','summary_6','summary_7','best_summary_index','best_summary_score'],axis=1, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   report     3047 non-null   object
 1   summary_2  3043 non-null   object
 2   summary_3  3042 non-null   object
 3   summary_1  3040 non-null   object
dtypes: object(4)
memory usage: 95.3+ KB


In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df = df[['report','summary_1', 'summary_2', 'summary_3']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3031 entries, 0 to 3046
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   report     3031 non-null   object
 1   summary_1  3031 non-null   object
 2   summary_2  3031 non-null   object
 3   summary_3  3031 non-null   object
dtypes: object(4)
memory usage: 118.4+ KB


In [ ]:
# Combine summaries into a single text for each report
df['combined_summary'] = df[['summary_1', 'summary_2', 'summary_3']].apply(lambda x: ' '.join(x), axis=1)

# Use TF-IDF to vectorize and calculate cosine similarity between report and each summary
vectorizer = TfidfVectorizer()
report_tfidf = vectorizer.fit_transform(df['report'])
summary_tfidf = vectorizer.transform(df[['summary_1', 'summary_2', 'summary_3']].apply(lambda x: ' '.join(x), axis=1))

In [ ]:
# Calculate cosine similarity scores for each summary
cosine_similarities = cosine_similarity(report_tfidf, summary_tfidf)

# Use individual similarity scores as weights
weights = cosine_similarities.T  # Transpose to align with the dataframe structure

# Assign weights to each summary
df['weighted_summary_1'] = weights[0]
df['weighted_summary_2'] = weights[1]
df['weighted_summary_3'] = weights[2]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Specify the columns you want to normalize
columns_to_normalize = ['weighted_summary_1', 'weighted_summary_2', 'weighted_summary_3']

# Normalize the specified columns such that the sum of each row is equal to 1
df[columns_to_normalize] = df[columns_to_normalize].div(df[columns_to_normalize].sum(axis=1), axis=0)

In [ ]:
df.head()

,report,summary_1,summary_2,summary_3,combined_summary,weighted_summary_1,weighted_summary_2,weighted_summary_3
0,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,24303.04 10 November 2015 12:43 PM proof...,0.362103,0.308697,0.329200
1,Synergy Health plc \nAnnual Report and Accou...,Strategic report \nChief Executive's strategi...,2010 2011 2012 2013 2014\n286.4 287.3\n312.0\...,Synergy remains well placed \nto exploit its...,Strategic report \nChief Executive's strategi...,0.286676,0.369690,0.343634
2,Shanks Group plc Annual Report and Accounts 2...,4 shanks. annual report 2008\nI joined as Gro...,2 shanks. annual report 2008\nfinancial highl...,shanks. annual report 2008 3\nchairman’s stat...,4 shanks. annual report 2008\nI joined as Gro...,0.283412,0.315729,0.400860
3,Annual Report & Accounts\n2012 Contents\nWynn...,10 www.wynnstay.co.uk\nINTRODUCTION \nThe Gro...,1 Wynnstay Group Plc Annual Report & Accounts...,6 www.wynnstay.co.uk\nOVERVIEW\nIn my first s...,10 www.wynnstay.co.uk\nINTRODUCTION \nThe Gro...,0.303700,0.334223,0.362077
4,HSBC Holdings plc \nAnnual Report and Account...,7\nOverview Operating & Financial Review Gove...,HSBC HOLDINGS PLC \nReport of the Directors: ...,HSBC HOLDINGS PLC \nReport of the Directors: ...,7\nOverview Operating & Financial Review Gove...,0.296568,0.343151,0.360281


In [ ]:
new_df=df[:1000]

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df, open('/content/drive/MyDrive/NLP_Project/new_df1000.pkl', 'wb'))